In [1]:
import  pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter

In [2]:
pwd

'/home/jovyan/用户聚类-click'

In [3]:
df = pd.read_csv(r'/home/jovyan/click_action_path_raw.csv')

In [4]:
df.head()

,active.user_id,log.bury_id,click.description,bury_time,rnk
0,71325658,click_01_17_002,D4,2021/6/1 00:19,1
1,71325658,click_08_09_001,A30,2021/6/1 00:20,2
2,71325658,click_08_07_015,A16,2021/6/1 00:21,3
3,71325658,click_01_18_001,D6,2021/6/2 09:39,4
4,71325658,click_08_09_001,A30,2021/6/2 09:39,5


In [5]:
df.pivot(index='active.user_id',columns='rnk',values='click.description').head(10)

rnk,1,2,3,4,5,6,7,8,9,10,...,220,221,222,223,224,225,226,227,228,229
active.user_id,,,,,,,,,,,,,,,,,,,,,
71325658,D4,A30,A16,D6,A30,A2,A30,A30,A2,A2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71325671,D4,A30,A30,A37,B10,F1,D6,B10,A32,A30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71325754,A57,E1,B10,A32,A30,A7,A31,A30,A7,A21,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71325794,D4,B10,b15,A32,A30,E1,A32,B10,b14,B13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71325820,D4,B10,A32,A30,A30,A49,A31,A30,A30,A31,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71325839,D4,B10,B10,E1,E1,B10,E1,A32,E1,E1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71326239,D4,c5,F1,B10,A32,F1,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71326439,D4,A42,F1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71326780,D4,A58,A59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df1 = df.pivot(index='active.user_id',columns='rnk',values='click.description')
df1.describe()

rnk,1,2,3,4,5,6,7,8,9,10,...,220,221,222,223,224,225,226,227,228,229
count,11334,11229,11104,10925,10751,10550,10286,10019,9719,9365,...,1,1,1,1,1,1,1,1,1,1
unique,66,76,83,84,88,90,84,89,89,84,...,1,1,1,1,1,1,1,1,1,1
top,D4,A30,A30,A32,A32,A32,A30,A30,A32,A30,...,A1,A9,A16,A1,A7,A2,A2,A2,A2,A1
freq,6326,2863,1384,1290,1324,1216,1136,1078,988,958,...,1,1,1,1,1,1,1,1,1,1


In [7]:
df1[df1.index==71325658]

rnk,1,2,3,4,5,6,7,8,9,10,...,220,221,222,223,224,225,226,227,228,229
active.user_id,,,,,,,,,,,,,,,,,,,,,
71325658,D4,A30,A16,D6,A30,A2,A30,A30,A2,A2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
series = df1[df1.columns[0:]].apply(
    lambda x: ''.join(x.dropna().astype(str)),
    axis=1
)

In [9]:
series.head()

active.user_id
71325658    D4A30A16D6A30A2A30A30A2A2A58A30A2A2A9A9A9A30A2...
71325671    D4A30A30A37B10F1D6B10A32A30A31A30A16A9A9A9A9A9...
71325754         A57E1B10A32A30A7A31A30A7A21A7A49A50A30A7A2D6
71325794    D4B10b15A32A30E1A32B10b14B13B16b14B15D6A30A14A...
71325820                     D4B10A32A30A30A49A31A30A30A31A30
dtype: object

In [10]:
series.index

Int64Index([71325658, 71325671, 71325754, 71325794, 71325820, 71325839,
            71326239, 71326439, 71326780, 71326965,
            ...
            72106653, 72106669, 72106762, 72106793, 72106808, 72106840,
            72106902, 72106913, 72106991, 72107059],
           dtype='int64', name='active.user_id', length=11334)

## Get n-Gram

In [11]:
import re
def get_ngrams(query, n):
    tempQuery = str(query)
    a=re.findall('([A-z][0-9]?[0-9])',tempQuery)
    ngrams = []
    for i in range(0,len(a)-(n-1)):
        ngrams.append("".join(a[i:i+n]))
    return ngrams

In [12]:
gram = []
for i in series.index:
    ngrams = get_ngrams(series[i],4) ##change n-gram!!
    gram.append(ngrams)
series_3gram = pd.Series(gram,index=series.index)

In [13]:
d={}
for i in series.index:
    c = dict(Counter(series_3gram[i]))
    total = ''
    for key in c.keys():
        part = str(key)+'('+str(c[key])+')'
        total += part
    d[i]=total

In [14]:
list(d.items())[:2]

[(71325658,
  'D4A30A16D6(1)A30A16D6A30(1)A16D6A30A2(1)D6A30A2A30(1)A30A2A30A30(1)A2A30A30A2(1)A30A30A2A2(1)A30A2A2A58(1)A2A2A58A30(1)A2A58A30A2(1)A58A30A2A2(1)A30A2A2A9(1)A2A2A9A9(1)A2A9A9A9(1)A9A9A9A30(1)A9A9A30A2(1)A9A30A2A2(1)A30A2A2A1(1)A2A2A1c12(1)A2A1c12A30(1)A1c12A30A2(1)c12A30A2A9(1)A30A2A9A9(1)A2A9A9B10(1)A9A9B10b15(1)A9B10b15b14(1)B10b15b14B12(1)b15b14B12b31(1)'),
 (71325671,
  'D4A30A30A37(1)A30A30A37B10(1)A30A37B10F1(1)A37B10F1D6(1)B10F1D6B10(1)F1D6B10A32(1)D6B10A32A30(1)B10A32A30A31(1)A32A30A31A30(1)A30A31A30A16(1)A31A30A16A9(1)A30A16A9A9(1)A16A9A9A9(1)A9A9A9A9(6)A9A9A9c13(1)A9A9c13c5(1)A9c13c5C5(1)c13c5C5C5(1)c5C5C5C5(1)C5C5C5C5(2)C5C5C5A21(1)C5C5A21A21(1)C5A21A21A59(1)A21A21A59A41(1)A21A59A41A41(1)A59A41A41B10(1)A41A41B10b17(1)A41B10b17B3(1)B10b17B3A32(1)b17B3A32B10(1)B3A32B10A32(1)A32B10A32A31(1)B10A32A31A30(1)A32A31A30A9(1)A31A30A9c13(1)A30A9c13A9(1)A9c13A9A9(1)c13A9A9A9(1)A9A9A9c15(1)A9A9c15c15(1)A9c15c15B10(1)c15c15B10b17(1)c15B10b17B30(1)B10b17B30A32(1)b17B30A32A30

## Remove NAs in n-grams

In [15]:
droplist=[]
for i,j in d.items():
    grams=re.findall('([A-z][0-9]?[0-9])',j)
    if len(grams)<5: ##change n-gram here!!!!!
        droplist.append(i)
len(droplist)        

586

In [16]:
d1=d
for i in list(d1.keys()):
    if i in droplist:
        del d1[i]

In [17]:
len(d1)

10748

In [18]:
#dir(pd.DataFrame)

In [19]:
d2=pd.DataFrame(d1.items(), columns=['user_id', 'seq']).reset_index()
d2['index_1']=d2['index']+1
d2.head()

,index,user_id,seq,index_1
0,0,71325658,D4A30A16D6(1)A30A16D6A30(1)A16D6A30A2(1)D6A30A...,1
1,1,71325671,D4A30A30A37(1)A30A30A37B10(1)A30A37B10F1(1)A37...,2
2,2,71325754,A57E1B10A32(1)E1B10A32A30(1)B10A32A30A7(1)A32A...,3
3,3,71325794,D4B10b15A32(1)B10b15A32A30(1)b15A32A30E1(1)A32...,4
4,4,71325820,D4B10A32A30(1)B10A32A30A30(1)A32A30A30A49(1)A3...,5


In [41]:
indexlist=[19, 21, 27, 32, 33, 38, 39, 107, 126, 137, 147, 149, 187, 214, 235, 252, 255, 284, 292, 296, 300, 319, 337, 346, 356, 385, 391, 393, 397, 429, 459, 480, 512, 520, 526, 540, 544, 562, 564, 570, 599, 656, 658, 660, 666, 674, 709, 713, 746, 750, 754, 803, 815, 817, 821, 825, 839, 844, 901, 994, 996, 1009, 1019, 1115, 1117, 1119, 1145, 1150, 1168, 1169, 1178, 1182, 1192, 1235, 1252, 1259, 1273, 1281, 1282, 1297, 1306, 1370, 1375, 1381, 1393, 1394, 1437, 1441, 1470, 1543, 1593, 1598, 1607, 1616, 1630, 1668, 1691, 1750, 1813, 1821, 1868, 1898, 1914, 1922, 1939, 1971, 2007, 2017, 2018, 2031, 2042, 2062, 2093, 2127, 2180, 2204, 2218, 2259, 2303, 2315, 2404, 2416, 2439, 2525, 2532, 2570, 2606, 2650, 2665, 2686, 2698, 2715, 2716, 2771, 2776, 2784, 2834, 2873, 2904, 2936, 2938, 2939, 2965, 3008, 3011, 3080, 3099, 3110, 3116, 3124, 3171, 3185, 3223, 3241, 3264, 3299, 3301, 3302, 3358, 3380, 3381, 3390, 3463, 3492, 3514, 3552, 3559, 3567, 3571, 3592, 3634, 3649, 3676, 3695, 3752, 3753, 3816, 3842, 3846, 3860, 3879, 3880, 3960, 3967, 4001, 4002, 4014, 4017, 4022, 4055, 4071, 4170, 4211, 4258, 4266, 4286, 4301, 4304, 4353, 4363, 4400, 4532, 4545, 4547, 4548, 4552, 4576, 4587, 4599, 4608, 4612, 4661, 4664, 4679, 4703, 4730, 4777, 4800, 4820, 4855, 4860, 4861, 4878, 4896, 4916, 4918, 4929, 4936, 4949, 4969, 4986, 5012, 5013, 5020, 5053, 5059, 5066, 5094, 5103, 5136, 5137, 5156, 5174, 5184, 5281, 5301, 5302, 5317, 5328, 5357, 5376, 5392, 5416, 5420, 5439, 5440, 5465, 5468, 5491, 5502, 5575, 5609, 5618, 5670, 5691, 5702, 5708, 5710, 5781, 5872, 5883, 5886, 5944, 5951, 5967, 6026, 6039, 6082, 6147, 6162, 6165, 6209, 6267, 6291, 6311, 6363, 6373, 6382, 6403, 6409, 6472, 6575, 6586, 6601, 6602, 6607, 6657, 6676, 6702, 6731, 6750, 6751, 6753, 6773, 6777, 6831, 6857, 6872, 6886, 6889, 6937, 7013, 7034, 7074, 7137, 7160, 7173, 7174, 7191, 7195, 7255, 7291, 7295, 7318, 7324, 7343, 7376, 7426, 7460, 7500, 7503, 7516, 7549, 7552, 7585, 7608, 7622, 7625, 7638, 7657, 7685, 7687, 7691, 7693, 7702, 7711, 7719, 7746, 7776, 7822, 7825, 7866, 7890, 7912, 7934, 7937, 7950, 7965, 8110, 8125, 8139, 8159, 8180, 8233, 8247, 8259, 8307, 8330, 8356, 8367, 8380, 8470, 8478, 8495, 8503, 8514, 8550, 8554, 8589, 8627, 8664, 8673, 8748, 8785, 8787, 8832, 8835, 8850, 8866, 8871, 8908, 8925, 8931, 8940, 8955, 9011, 9029, 9065, 9128, 9156, 9170, 9208, 9266, 9295, 9361, 9390, 9396, 9412, 9472, 9522, 9532, 9556, 9564, 9580, 9590, 9625, 9652, 9655, 9673, 9703, 9726, 9759, 9776, 9788, 9825, 9827, 9833, 9866, 9882, 9893, 9901, 9942, 9947, 9949, 9978, 9993, 10033, 10052, 10090, 10103, 10112, 10144, 10163, 10165, 10166, 10175, 10181, 10182, 10190, 10237, 10243, 10246, 10259, 10274, 10275, 10289, 10303, 10306, 10307, 10317, 10392, 10407, 10422, 10482, 10495, 10501, 10506, 10542, 10546, 10547, 10573, 10586, 10589, 10640, 10685, 10702, 10707, 10709, 10725, 9, 26, 42, 47, 90, 96, 117, 123, 134, 141, 150, 166, 206, 219, 271, 371, 390, 448, 471, 483, 484, 527, 563, 572, 574, 606, 612, 622, 650, 663, 667, 688, 699, 719, 730, 732, 737, 809, 810, 829, 860, 891, 902, 916, 926, 946, 947, 951, 960, 965, 981, 1007, 1015, 1021, 1051, 1054, 1079, 1087, 1093, 1108, 1123, 1138, 1149, 1199, 1212, 1218, 1219, 1220, 1254, 1255, 1269, 1270, 1276, 1279, 1283, 1294, 1305, 1312, 1324, 1343, 1376, 1378, 1384, 1406, 1412, 1416, 1419, 1424, 1458, 1467, 1609, 1636, 1671, 1672, 1675, 1714, 1742, 1752, 1771, 1784, 1786, 1825, 1842, 1844, 1881, 1895, 1910, 1915, 1931, 1950, 1984, 2006, 2008, 2020, 2052, 2069, 2094, 2104, 2115, 2172, 2177, 2178, 2193, 2227, 2243, 2246, 2266, 2275, 2299, 2301, 2322, 2378, 2417, 2453, 2468, 2511, 2530, 2543, 2547, 2560, 2567, 2598, 2617, 2629, 2631, 2638, 2690, 2740, 2793, 2887, 2945, 3002, 3009, 3018, 3024, 3069, 3073, 3129, 3130, 3141, 3187, 3218, 3282, 3286, 3305, 3391, 3398, 3399, 3471, 3480, 3488, 3574, 3646, 3654, 3688, 3770, 3772, 3797, 3808, 3834, 3849, 3877, 3878, 3913, 3922, 3941, 3956, 3964, 3993, 4008, 4074, 4091, 4099, 4145, 4173, 4193, 4226, 4274, 4288, 4321, 4338, 4346, 4349, 4350, 4387, 4435, 4436, 4442, 4495, 4513, 4526, 4528, 4554, 4604, 4605, 4616, 4619, 4672, 4683, 4697, 4781, 4793, 4815, 4886, 4894, 4897, 4911, 4930, 4957, 4996, 5007, 5037, 5089, 5118, 5125, 5141, 5175, 5203, 5215, 5233, 5236, 5267, 5270, 5284, 5407, 5409, 5426, 5431, 5435, 5438, 5441, 5453, 5455, 5503, 5504, 5551, 5615, 5623, 5687, 5688, 5787, 5798, 5800, 5826, 5873, 5887, 5914, 5958, 5980, 6014, 6033, 6105, 6148, 6151, 6174, 6189, 6196, 6243, 6245, 6275, 6288, 6341, 6447, 6452, 6539, 6544, 6571, 6596, 6630, 6651, 6655, 6674, 6683, 6720, 6746, 6821, 6830, 6855, 6882, 6985, 6993, 7002, 7006, 7012, 7066, 7079, 7085, 7095, 7152, 7162, 7166, 7178, 7248, 7260, 7266, 7308, 7368, 7388, 7405, 7415, 7439, 7440, 7470, 7483, 7485, 7508, 7509, 7533, 7562, 7569, 7601, 7602, 7605, 7667, 7683, 7717, 7720, 7737, 7764, 7779, 7787, 7791, 7826, 7839, 7858, 7869, 7892, 7898, 7930, 7933, 7982, 7986, 7994, 8021, 8031, 8046, 8069, 8126, 8144, 8156, 8158, 8175, 8268, 8332, 8386, 8387, 8390, 8447, 8494, 8519, 8525, 8549, 8553, 8568, 8594, 8595, 8597, 8601, 8611, 8620, 8631, 8647, 8694, 8716, 8723, 8724, 8736, 8807, 8811, 8826, 8858, 8864, 8876, 8894, 8898, 8902, 8905, 8909, 8932, 8963, 8988, 8989, 9024, 9037, 9103, 9118, 9131, 9152, 9191, 9207, 9227, 9254, 9263, 9272, 9313, 9332, 9381, 9389, 9403, 9408, 9424, 9432, 9433, 9436, 9453, 9464, 9465, 9468, 9484, 9551, 9616, 9635, 9662, 9671, 9674, 9678, 9683, 9685, 9696, 9725, 9761, 9764, 9773, 9781, 9782, 9792, 9804, 9810, 9817, 9855, 9859, 9926, 9931, 9957, 10035, 10072, 10085, 10120, 10135, 10176, 10202, 10209, 10219, 10224, 10236, 10242, 10260, 10290, 10318, 10319, 10356, 10372, 10394, 10397, 10424, 10466, 10471, 10488, 10498, 10500, 10529, 10530, 10537, 10543, 10551, 10567, 10571, 10572, 10598, 10614, 10622, 10633, 10691, 10724, 10733, 10736, 24, 71, 92, 102, 128, 175, 190, 195, 200, 203, 220, 241, 290, 317, 341, 365, 368, 382, 411, 423, 435, 439, 444, 579, 588, 626, 632, 640, 647, 686, 692, 749, 804, 841, 875, 885, 886, 933, 1034, 1103, 1122, 1140, 1173, 1316, 1331, 1341, 1377, 1399, 1408, 1422, 1439, 1440, 1468, 1494, 1499, 1508, 1522, 1536, 1541, 1542, 1551, 1553, 1565, 1601, 1622, 1680, 1721, 1730, 1732, 1740, 1744, 1766, 1768, 1770, 1809, 1826, 1833, 1849, 1891, 1893, 1894, 1917, 2028, 2029, 2035, 2073, 2074, 2085, 2088, 2106, 2129, 2159, 2169, 2186, 2199, 2214, 2254, 2261, 2323, 2328, 2353, 2379, 2449, 2462, 2497, 2538, 2602, 2607, 2608, 2610, 2700, 2749, 2765, 2828, 2856, 2917, 2921, 2922, 2929, 2949, 2950, 2978, 2986, 2987, 3000, 3017, 3021, 3046, 3064, 3101, 3104, 3114, 3137, 3175, 3195, 3207, 3222, 3231, 3259, 3290, 3292, 3296, 3414, 3496, 3508, 3513, 3540, 3551, 3569, 3572, 3579, 3585, 3588, 3598, 3599, 3635, 3674, 3675, 3703, 3721, 3787, 3812, 3852, 3865, 3949, 3969, 3972, 3997, 4010, 4029, 4035, 4076, 4102, 4107, 4112, 4138, 4204, 4235, 4247, 4283, 4306, 4384, 4423, 4432, 4433, 4440, 4446, 4460, 4468, 4475, 4491, 4500, 4504, 4610, 4673, 4675, 4727, 4738, 4745, 4799, 4858, 4893, 4905, 4910, 4937, 4941, 4952, 4965, 5016, 5046, 5121, 5132, 5135, 5168, 5198, 5216, 5231, 5261, 5312, 5313, 5316, 5323, 5333, 5337, 5418, 5421, 5448, 5459, 5488, 5490, 5521, 5545, 5584, 5602, 5607, 5628, 5633, 5657, 5711, 5723, 5748, 5791, 5793, 5821, 5852, 5882, 5891, 5892, 5908, 5923, 5925, 5988, 5992, 6000, 6003, 6011, 6046, 6052, 6054, 6064, 6112, 6121, 6125, 6145, 6177, 6187, 6208, 6239, 6261, 6283, 6302, 6306, 6312, 6325, 6354, 6355, 6477, 6510, 6515, 6517, 6534, 6540, 6561, 6578, 6627, 6635, 6661, 6667, 6672, 6679, 6681, 6691, 6726, 6801, 6823, 6835, 6881, 6897, 6926, 6983, 6990, 7025, 7057, 7060, 7070, 7096, 7107, 7164, 7168, 7203, 7205, 7215, 7261, 7276, 7292, 7322, 7332, 7393, 7431, 7450, 7501, 7529, 7539, 7587, 7591, 7633, 7651, 7677, 7680, 7738, 7741, 7757, 7810, 7840, 7860, 7881, 7901, 7948, 8008, 8068, 8078, 8082, 8100, 8104, 8120, 8127, 8161, 8165, 8171, 8207, 8252, 8271, 8308, 8391, 8413, 8435, 8437, 8461, 8498, 8509, 8530, 8560, 8572, 8618, 8633, 8635, 8695, 8702, 8789, 8875, 8889, 8942, 8997, 9013, 9039, 9044, 9053, 9074, 9095, 9096, 9107, 9111, 9121, 9134, 9146, 9147, 9151, 9164, 9178, 9220, 9242, 9248, 9275, 9300, 9302, 9311, 9326, 9328, 9348, 9353, 9374, 9423, 9482, 9486, 9492, 9563, 9575, 9611, 9648, 9653, 9654, 9672, 9691, 9712, 9731, 9754, 9767, 9770, 9787, 9820, 9856, 9874, 9911, 9924, 9928, 9953, 9962, 9975, 10047, 10118, 10250, 10279, 10325, 10331, 10332, 10342, 10355, 10362, 10374, 10423, 10431, 10467, 10473, 10477, 10497, 10514, 10541, 10549, 10566, 10577, 10639, 10701, 10732,185, 192, 212, 221, 330, 404, 424, 498, 528, 530, 728, 831, 1035, 1052, 1129, 1157, 1237, 1308, 1320, 1325, 1365, 1372, 1389, 1395, 1396, 1402, 1426, 1454, 1481, 1501, 1506, 1525, 1531, 1533, 1552, 1572, 1583, 1588, 1666, 1711, 1731, 1759, 1799, 1829, 1871, 1924, 1929, 1934, 1974, 2013, 2014, 2044, 2151, 2223, 2258, 2264, 2267, 2288, 2337, 2430, 2544, 2553, 2566, 2574, 2661, 2789, 2808, 2864, 2866, 2948, 2966, 2970, 2995, 2996, 3004, 3027, 3033, 3047, 3134, 3166, 3266, 3387, 3459, 3483, 3497, 3506, 3526, 3539, 3581, 3656, 3796, 3800, 3840, 3881, 3904, 3931, 3962, 4012, 4241, 4275, 4280, 4315, 4396, 4397, 4399, 4421, 4425, 4457, 4521, 4557, 4564, 4647, 4655, 4686, 4714, 4774, 4925, 4966, 5022, 5043, 5143, 5187, 5256, 5326, 5367, 5389, 5394, 5415, 5449, 5486, 5536, 5561, 5664, 5680, 5721, 5732, 5742, 5799, 5801, 5812, 5830, 5832, 5845, 5916, 5921, 5946, 5974, 5987, 6008, 6022, 6028, 6042, 6061, 6068, 6071, 6089, 6150, 6289, 6310, 6313, 6396, 6400, 6565, 6577, 6589, 6593, 6604, 6626, 6641, 6647, 6648, 6662, 6682, 6748, 6798, 6827, 6845, 7019, 7030, 7187, 7228, 7229, 7257, 7275, 7281, 7311, 7313, 7371, 7386, 7626, 7652, 7654, 7701, 7730, 7749, 7941, 7991, 8039, 8173, 8221, 8369, 8382, 8443, 8445, 8471, 8520, 8531, 8536, 8551, 8578, 8596, 8692, 8705, 8740, 8741, 8755, 8934, 8936, 8946, 8996, 9033, 9051, 9077, 9182, 9188, 9221, 9251, 9287, 9291, 9399, 9420, 9438, 9440, 9444, 9480, 9487, 9497, 9547, 9555, 9623, 9670, 9694, 9708, 9710, 9757, 9769, 9790, 9801, 9829, 9867, 9871, 9922, 9934, 10075, 10124, 10150, 10233, 10248, 10366, 10464, 10487, 10490, 10527, 10544, 10548, 10653, 10741,6, 10, 13, 17, 86, 91, 114, 173, 208, 395, 407, 430, 445, 501, 509, 584, 786, 835, 870, 912, 945, 953, 998, 999, 1008, 1042, 1082, 1088, 1113, 1251, 1262, 1301, 1311, 1328, 1330, 1336, 1360, 1361, 1405, 1448, 1495, 1526, 1537, 1568, 1600, 1696, 1772, 1796, 1937, 1993, 2002, 2016, 2050, 2067, 2086, 2097, 2131, 2152, 2190, 2203, 2231, 2234, 2236, 2331, 2358, 2369, 2451, 2483, 2498, 2548, 2561, 2601, 2618, 2658, 2742, 2876, 2923, 2932, 2988, 2989, 2992, 3007, 3038, 3041, 3057, 3092, 3225, 3273, 3434, 3437, 3472, 3475, 3491, 3498, 3502, 3511, 3583, 3593, 3611, 3673, 3765, 3781, 3804, 3831, 3833, 3918, 3919, 4114, 4144, 4216, 4307, 4328, 4445, 4453, 4487, 4489, 4490, 4520, 4550, 4579, 4618, 4639, 4749, 4757, 4798, 4853, 4866, 4958, 4967, 5024, 5127, 5176, 5213, 5251, 5290, 5303, 5305, 5338, 5350, 5353, 5355, 5362, 5368, 5384, 5387, 5390, 5414, 5419, 5475, 5509, 5552, 5586, 5597, 5705, 5720, 5744, 5763, 5815, 5877, 5900, 5935, 5956, 5959, 6019, 6025, 6030, 6040, 6057, 6101, 6118, 6132, 6144, 6167, 6234, 6240, 6263, 6293, 6342, 6357, 6399, 6404, 6460, 6476, 6479, 6524, 6528, 6541, 6590, 6649, 6664, 6677, 6718, 6723, 6805, 6861, 6925, 6958, 7000, 7185, 7217, 7251, 7284, 7285, 7297, 7303, 7304, 7363, 7370, 7423, 7472, 7627, 7635, 7695, 7734, 7747, 7758, 7854, 7955, 7974, 8004, 8011, 8026, 8059, 8067, 8072, 8079, 8090, 8137, 8143, 8148, 8150, 8238, 8240, 8258, 8266, 8368, 8370, 8468, 8485, 8487, 8504, 8538, 8548, 8583, 8609, 8772, 8838, 8886, 8966, 8992, 8995, 9001, 9031, 9043, 9122, 9206, 9296, 9307, 9308, 9314, 9321, 9342, 9356, 9401, 9416, 9450, 9496, 9554, 9568, 9572, 9584, 9676, 9699, 9715, 9717, 9729, 9736, 9740, 9823, 9844, 9862, 9910, 9915, 9963, 9980, 10012, 10049, 10083, 10126, 10137, 10187, 10203, 10239, 10298, 10365, 10427, 10458, 10484, 10486, 10531, 10555, 10558, 10595, 10657, 10686, 10697]

In [42]:
users=d2[d2['index_1'].isin(indexlist)]
ids=list(users['user_id'])

In [43]:
mydf=df1.reset_index()
mydf.head()

rnk,active.user_id,1,2,3,4,5,6,7,8,9,...,220,221,222,223,224,225,226,227,228,229
0,71325658,D4,A30,A16,D6,A30,A2,A30,A30,A2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,71325671,D4,A30,A30,A37,B10,F1,D6,B10,A32,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,71325754,A57,E1,B10,A32,A30,A7,A31,A30,A7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,71325794,D4,B10,b15,A32,A30,E1,A32,B10,b14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,71325820,D4,B10,A32,A30,A30,A49,A31,A30,A30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
mydf1=mydf[mydf['active.user_id'].isin(ids)].reset_index()
mydf1.head()

rnk,index,active.user_id,1,2,3,4,5,6,7,8,...,220,221,222,223,224,225,226,227,228,229
0,5,71325839,D4,B10,B10,E1,E1,B10,E1,A32,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10,71326976,B10,b20,b17,A32,A30,F1,E1,B10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,11,71327017,D4,B10,A32,E1,A32,F1,A32,A41,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,14,71327221,D4,A49,D6,A46,A46,E1,A32,E1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,18,71327363,B10,b16,E1,A32,A24,A24,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
code={'D4':'选择年级','B10':'我的','b15':'个人资料','A32':'首页','A30':'首页球区',
      'E1':'学习','F1':'发现','b14':'我的设置','B12':'账号与安全','A2':'同步课筛选',
      'A7':'学科导航','A14':'一行课程包','A1':'同步课课程卡片','c14':'课程详情页直播大纲查看所有',
     'A57':'特训班去报名','D1':'隐私协议勾选','A31':'首页年级选择','A49':'首页球区全部icon',
      'A50':'首页球区二级页面快链','A21':'学科导航','A58':'爆品课/精品课0元预约','A37':'学习规划师',
     'D6':'隐私协议同意','A41':'点击非转化类正价课','A36':'了解网校','b31':'退出当前账号',
     'A46':'底部学习规划师','B17':'注销账号','b17':'我的订单','A24':'信息流卡片','A59':'品牌名师带课立即报名',
     'A42':'首页_品牌1-3号位','D5':'选择地区','b20':'点击赠课卡','b16':'点击金币商城'}
codedf=mydf1.replace(code)
codedf.head()

rnk,index,active.user_id,1,2,3,4,5,6,7,8,...,220,221,222,223,224,225,226,227,228,229
0,5,71325839,选择年级,我的,我的,学习,学习,我的,学习,首页,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10,71326976,我的,点击赠课卡,我的订单,首页,首页球区,发现,学习,我的,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,11,71327017,选择年级,我的,首页,学习,首页,发现,首页,点击非转化类正价课,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,14,71327221,选择年级,首页球区全部icon,隐私协议同意,底部学习规划师,底部学习规划师,学习,首页,学习,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,18,71327363,我的,点击金币商城,学习,首页,信息流卡片,信息流卡片,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
series_1 = codedf[codedf.columns[1:]].apply(
    lambda x: ' '.join(x.dropna().astype(str)),
    axis=1
)
series_1

0       71325839 选择年级 我的 我的 学习 学习 我的 学习 首页 学习 学习 首页 学习 我的
1       71326976 我的 点击赠课卡 我的订单 首页 首页球区 发现 学习 我的 B3 B30...
2       71327017 选择年级 我的 首页 学习 首页 发现 首页 点击非转化类正价课 首页球区...
3       71327221 选择年级 首页球区全部icon 隐私协议同意 底部学习规划师 底部学习规划...
4                    71327363 我的 点击金币商城 学习 首页 信息流卡片 信息流卡片
5       71327574 首页 我的 点击赠课卡 首页 我的 点击赠课卡 点击赠课卡 点击赠课卡 首...
6            71327736 选择地区 选择年级 我的 个人资料 首页 我的 首页 首页球区 A11
7                        71328302 我的 个人资料 个人资料 学习 首页 首页球区
8       71328374 选择年级 首页 首页年级选择 我的 我的设置 账号与安全 注销账号 账号与...
9       71328524 首页 我的 我的设置 b22 点击金币商城 点击金币商城 首页 隐私协议同...
10      71328979 选择地区 选择年级 我的 我的设置 B16 账号与安全 注销账号 D7 隐...
11      71329022 首页 首页球区 特训班去报名 我的 我的设置 b22 b36 b33 B1...
12      71329185 选择年级 底部学习规划师 隐私协议同意 我的 个人资料 首页 我的 我的设...
13      71329191 首页 首页_品牌1-3号位 首页球区 首页_品牌1-3号位 我的 我的设置...
14      71329213 选择年级 我的 首页 我的 B2 隐私协议同意 我的 B2 发现 首页 首...
15      71329441 选择年级 我的 首页 学习 我的 点击赠课卡 发现 我的 点击赠课卡 学习...
16      71331466 选择年级 隐私协议同意 我的 我的订单 首页 首页球区 学习 我的 点击赠...
17      713331

In [51]:
type(series_1)

pandas.core.series.Series

In [52]:
series_1.to_csv("users.txt",index=False)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.
